In [1]:
import pandas as pd
import os
import json
from collections import OrderedDict
import re
import numpy as np

In [2]:
mapping = OrderedDict({    
    'model=BaseModel,network=resnet152,percent_on_k_winner=0.25': 'Resnet-Dense-Kwinners',
    'model=SparseModel,network=resnet152,percent_on_k_winner=0.25': 'Resnet-Sparse-Kwinners',
    'model=BaseModel,network=WideResNet,percent_on_k_winner=0.25': 'WideResnet-Dense-Kwinners',
    'model=SparseModel,network=WideResNet,percent_on_k_winner=0.25': 'WideResnet-Sparse-Kwinners',
    'model=BaseModel,network=resnet152,percent_on_k_winner=1': 'Resnet-Dense-ReLU',
    'model=SparseModel,network=resnet152,percent_on_k_winner=1': 'Resnet-Sparse-ReLU',
    'model=BaseModel,network=WideResNet,percent_on_k_winner=1': 'WideResnet-Dense-ReLU',
    'model=SparseModel,network=WideResNet,percent_on_k_winner=1': 'WideResnet-Sparse-ReLU',
})

In [3]:
# a = '23_model=BaseModel,network=resnet152,percent_on_k_winner=0.25'
# a = re.sub(r'^\d+_', '', a)
# a

### Run 1

In [11]:
file = os.path.expanduser("~/nta/results/resnet_cifar3/noise_results.json")
with open(file, 'r') as f:
    noise_results = json.load(f)

mapped_results = {}
for k,v in noise_results.items():
    # remove numbering
    k = re.sub(r'^\d+_', '', k)
    new_k = mapping[k]
    if new_k not in mapped_results:
        mapped_results[new_k] = {}
    for noise_level, acc in v.items():
        if noise_level not in mapped_results[new_k]:
            mapped_results[new_k][noise_level] = [acc]
        else:
            mapped_results[new_k][noise_level].append(acc)

# average over
for k, v in mapped_results.items():
    for noise_level, accuracies in v.items():
        mapped_results[k][noise_level] = "{:.2f} ± {:.2f}".format(np.mean(accuracies)*100, np.std(accuracies)*100)
# print(mapped_results)
                    
df = pd.DataFrame.from_dict(mapped_results)
df = df.drop('0.2')
df.columns

Index(['Resnet-Dense-Kwinners', 'Resnet-Sparse-ReLU',
       'WideResnet-Dense-Kwinners', 'WideResnet-Dense-ReLU',
       'WideResnet-Sparse-Kwinners', 'Resnet-Sparse-Kwinners',
       'Resnet-Dense-ReLU', 'WideResnet-Sparse-ReLU'],
      dtype='object')

In [12]:
df.head()

,Resnet-Dense-Kwinners,Resnet-Sparse-ReLU,WideResnet-Dense-Kwinners,WideResnet-Dense-ReLU,WideResnet-Sparse-Kwinners,Resnet-Sparse-Kwinners,Resnet-Dense-ReLU,WideResnet-Sparse-ReLU
0,93.25 ± 0.26,93.68 ± 0.27,94.72 ± 0.07,94.85 ± 0.13,94.08 ± 0.19,91.92 ± 0.20,93.52 ± 0.70,94.28 ± 0.10
0.025,88.15 ± 0.63,87.66 ± 0.51,85.59 ± 0.40,84.78 ± 0.43,85.26 ± 0.80,85.92 ± 0.63,86.79 ± 1.38,85.69 ± 0.41
0.05,79.68 ± 1.48,77.39 ± 0.99,74.44 ± 1.08,73.01 ± 0.63,74.45 ± 1.59,76.91 ± 1.51,76.73 ± 1.48,74.10 ± 0.91
0.075,69.36 ± 2.24,64.35 ± 1.25,61.82 ± 1.60,60.07 ± 0.88,61.61 ± 2.46,66.49 ± 1.79,64.91 ± 1.60,60.90 ± 2.20
0.1,59.36 ± 3.10,51.84 ± 1.67,49.34 ± 1.80,48.19 ± 1.41,49.09 ± 3.22,56.32 ± 1.82,53.23 ± 1.92,48.57 ± 3.16


In [14]:
# paper comparison table
df[['Resnet-Dense-ReLU', 'Resnet-Sparse-Kwinners', 'WideResnet-Dense-ReLU', 'WideResnet-Sparse-Kwinners']]

,Resnet-Dense-ReLU,Resnet-Sparse-Kwinners,WideResnet-Dense-ReLU,WideResnet-Sparse-Kwinners
0,93.52 ± 0.70,91.92 ± 0.20,94.85 ± 0.13,94.08 ± 0.19
0.025,86.79 ± 1.38,85.92 ± 0.63,84.78 ± 0.43,85.26 ± 0.80
0.05,76.73 ± 1.48,76.91 ± 1.51,73.01 ± 0.63,74.45 ± 1.59
0.075,64.91 ± 1.60,66.49 ± 1.79,60.07 ± 0.88,61.61 ± 2.46
0.1,53.23 ± 1.92,56.32 ± 1.82,48.19 ± 1.41,49.09 ± 3.22
0.125,43.38 ± 1.70,47.23 ± 1.90,38.23 ± 1.42,38.48 ± 3.38
0.15,35.67 ± 1.47,39.70 ± 1.86,31.06 ± 1.46,30.93 ± 3.38
0.175,30.03 ± 1.57,33.87 ± 1.79,25.72 ± 1.44,25.44 ± 3.10


In [15]:
# dense weights only
df[['Resnet-Dense-ReLU', 'Resnet-Dense-Kwinners', 'WideResnet-Dense-ReLU', 'WideResnet-Dense-Kwinners']]

,Resnet-Dense-ReLU,Resnet-Dense-Kwinners,WideResnet-Dense-ReLU,WideResnet-Dense-Kwinners
0,93.52 ± 0.70,93.25 ± 0.26,94.85 ± 0.13,94.72 ± 0.07
0.025,86.79 ± 1.38,88.15 ± 0.63,84.78 ± 0.43,85.59 ± 0.40
0.05,76.73 ± 1.48,79.68 ± 1.48,73.01 ± 0.63,74.44 ± 1.08
0.075,64.91 ± 1.60,69.36 ± 2.24,60.07 ± 0.88,61.82 ± 1.60
0.1,53.23 ± 1.92,59.36 ± 3.10,48.19 ± 1.41,49.34 ± 1.80
0.125,43.38 ± 1.70,50.20 ± 3.30,38.23 ± 1.42,38.82 ± 1.61
0.15,35.67 ± 1.47,42.81 ± 3.13,31.06 ± 1.46,31.15 ± 1.43
0.175,30.03 ± 1.57,36.68 ± 3.12,25.72 ± 1.44,25.65 ± 1.14


In [16]:
# sparse weights only
df[['Resnet-Sparse-ReLU', 'Resnet-Sparse-Kwinners', 'WideResnet-Sparse-ReLU', 'WideResnet-Sparse-Kwinners']]

,Resnet-Sparse-ReLU,Resnet-Sparse-Kwinners,WideResnet-Sparse-ReLU,WideResnet-Sparse-Kwinners
0,93.68 ± 0.27,91.92 ± 0.20,94.28 ± 0.10,94.08 ± 0.19
0.025,87.66 ± 0.51,85.92 ± 0.63,85.69 ± 0.41,85.26 ± 0.80
0.05,77.39 ± 0.99,76.91 ± 1.51,74.10 ± 0.91,74.45 ± 1.59
0.075,64.35 ± 1.25,66.49 ± 1.79,60.90 ± 2.20,61.61 ± 2.46
0.1,51.84 ± 1.67,56.32 ± 1.82,48.57 ± 3.16,49.09 ± 3.22
0.125,41.78 ± 1.93,47.23 ± 1.90,38.53 ± 3.61,38.48 ± 3.38
0.15,34.59 ± 2.10,39.70 ± 1.86,31.02 ± 3.60,30.93 ± 3.38
0.175,29.31 ± 2.14,33.87 ± 1.79,25.76 ± 3.27,25.44 ± 3.10


In [18]:
# relu only
df[['Resnet-Dense-ReLU', 'Resnet-Sparse-ReLU', 'WideResnet-Dense-ReLU', 'WideResnet-Sparse-ReLU']]

,Resnet-Dense-ReLU,Resnet-Sparse-ReLU,WideResnet-Dense-ReLU,WideResnet-Sparse-ReLU
0,93.52 ± 0.70,93.68 ± 0.27,94.85 ± 0.13,94.28 ± 0.10
0.025,86.79 ± 1.38,87.66 ± 0.51,84.78 ± 0.43,85.69 ± 0.41
0.05,76.73 ± 1.48,77.39 ± 0.99,73.01 ± 0.63,74.10 ± 0.91
0.075,64.91 ± 1.60,64.35 ± 1.25,60.07 ± 0.88,60.90 ± 2.20
0.1,53.23 ± 1.92,51.84 ± 1.67,48.19 ± 1.41,48.57 ± 3.16
0.125,43.38 ± 1.70,41.78 ± 1.93,38.23 ± 1.42,38.53 ± 3.61
0.15,35.67 ± 1.47,34.59 ± 2.10,31.06 ± 1.46,31.02 ± 3.60
0.175,30.03 ± 1.57,29.31 ± 2.14,25.72 ± 1.44,25.76 ± 3.27


In [19]:
# kwinners only
df[['Resnet-Dense-Kwinners', 'Resnet-Sparse-Kwinners', 'WideResnet-Dense-Kwinners', 'WideResnet-Sparse-Kwinners']]

,Resnet-Dense-Kwinners,Resnet-Sparse-Kwinners,WideResnet-Dense-Kwinners,WideResnet-Sparse-Kwinners
0,93.25 ± 0.26,91.92 ± 0.20,94.72 ± 0.07,94.08 ± 0.19
0.025,88.15 ± 0.63,85.92 ± 0.63,85.59 ± 0.40,85.26 ± 0.80
0.05,79.68 ± 1.48,76.91 ± 1.51,74.44 ± 1.08,74.45 ± 1.59
0.075,69.36 ± 2.24,66.49 ± 1.79,61.82 ± 1.60,61.61 ± 2.46
0.1,59.36 ± 3.10,56.32 ± 1.82,49.34 ± 1.80,49.09 ± 3.22
0.125,50.20 ± 3.30,47.23 ± 1.90,38.82 ± 1.61,38.48 ± 3.38
0.15,42.81 ± 3.13,39.70 ± 1.86,31.15 ± 1.43,30.93 ± 3.38
0.175,36.68 ± 3.12,33.87 ± 1.79,25.65 ± 1.14,25.44 ± 3.10
